# 1) Implementing neural networks from scratch



We will use sigmoid as the activation function in this example. We first define the sigmoid function and its derivative function:


In [ ]:
import numpy as np

def sigmoid(z):
  return 1.0/ (1 + np.exp(-z))
def sigmoid_derivative(z):
  return sigmoid(z) * (1.0 - sigmoid(z))

We then define the training function, which takes in the training dataset, the number of units in the hidden layer (we will only use one hidden layer as an example), and the number of iterations:

In [ ]:
def train(X, y, n_hidden, learning_rate, n_iter):
  m, n_input = X.shape
  W1 = np.random.randn(n_input, n_hidden)
  b1 = np.zeros((1, n_hidden))
  W2 = np.random.randn (n_hidden, 1)
  b2 = np.zeros((1, 1))
  for i in range(1, n_iter+1):
    Z2 = np.matmul(X, W1) + b1
    A2 = sigmoid (Z2)
    Z3 = np.matmul (A2, W2) + b2
    A3 = Z3
    dZ3 = A3 - y
    dW2 = np.matmul(A2.T, dZ3)
    db2 = np. sum(dZ3, axis=0, keepdims=True)
    dZ2 = np.matmul(dZ3, W2.T) * sigmoid_derivative(Z2)
    dW1 = np.matmul(X.T, dZ2)
    db1 = np.sum(dZ2, axis=0)
    W2 = W2 - learning_rate * dW2 / m
    b2 = b2 - learning_rate * db2 / m
    W1 = W1 - learning_rate * dW1 / m
    b1 = b1 - learning_rate * db1 / m
    if i % 100 == 0:
      cost = np.mean((y - A3) ** 2)
      print('Iteration %i, training loss: %f' % (i, cost))
  model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
  return model

To test the model, we will use Boston house prices as the toy dataset (link). As a reminder, data normalization is usually recommended whenever gradient descent is used. Hence, we will standardize the input data by removing the mean and scaling to unit variance:

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# getting dataset from original source as it is deprecated from scikit
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
num_test = 10 # the last 10 samples as testing set
scaler = preprocessing.StandardScaler()
X_train = data[:-num_test, :]
X_train = scaler.fit_transform(X_train)
y_train = target[:-num_test].reshape(-1, 1)
X_test = data[-num_test:, :]
X_test = scaler.transform(X_test)
y_test = target[-num_test:]

With the scaled dataset, we can now train a one-layer neural network with 20 hidden units, a 0.1 learning rate, and 2000 iterations:

In [ ]:
n_hidden = 20
learning_rate = 0.1
n_iter = 2000
model = train(X_train, y_train, n_hidden, learning_rate, n_iter)

Iteration 100, training loss: 12.179157
Iteration 200, training loss: 8.688119
Iteration 300, training loss: 7.329763
Iteration 400, training loss: 6.555287
Iteration 500, training loss: 5.999054
Iteration 600, training loss: 5.502339
Iteration 700, training loss: 4.921318
Iteration 800, training loss: 4.572597
Iteration 900, training loss: 4.318087
Iteration 1000, training loss: 4.103117
Iteration 1100, training loss: 3.887790
Iteration 1200, training loss: 3.701398
Iteration 1300, training loss: 3.543507
Iteration 1400, training loss: 3.407945
Iteration 1500, training loss: 3.292071
Iteration 1600, training loss: 3.192516
Iteration 1700, training loss: 3.105235
Iteration 1800, training loss: 3.026942
Iteration 1900, training loss: 2.955243
Iteration 2000, training loss: 2.888679


Then, we define a prediction function, which will take in a model and produce the regression results:

In [ ]:
def predict(x, model):
  W1 = model['W1']
  b1 = model['b1']
  W2 = model[ 'W2']
  b2 = model['b2']
  A2 = sigmoid(np.matmul(x, W1) + b1)
  A3 = np.matmul(A2, W2) + b2
  return A3

Finally, we apply the trained model on the testing set:


In [ ]:
predictions = predict(X_test, model)
print(predictions)
print(y_test)

[[18.903031  ]
 [19.9681165 ]
 [22.03546325]
 [20.1624136 ]
 [20.17966718]
 [25.85265925]
 [22.94359981]
 [29.51474843]
 [27.13594012]
 [21.61948489]]
[19.7 18.3 21.2 17.5 16.8 22.4 20.6 23.9 22.  11.9]


# 2) Implementing neural networks with scikit-learn

We will utilize the MLPRegressor class (MLP stands for multi-layer perceptron, a nickname for neural networks):

In [ ]:
from sklearn.neural_network import MLPRegressor
nn_scikit = MLPRegressor(hidden_layer_sizes=(16, 8),
                          activation='relu',
                          solver='adam',
                          learning_rate_init=0.001,
                          random_state=42, max_iter=2000)
nn_scikit.fit(X_train, y_train)
predictions = nn_scikit.predict(X_test)
print(predictions)
print(np.mean((y_test - predictions) ** 2))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[16.79582331 18.55538023 21.07961496 19.21362606 18.50955771 23.5608387
 22.27916529 27.11909153 24.70251262 22.05522035]
13.933482332708795


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)

model = keras.Sequential([
    keras.layers.Dense(units=20, activation='relu'),
    keras.layers.Dense(units=8, activation='relu'),
    keras.layers.Dense(units=1)
])
model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(0.02))
model.fit(X_train, y_train, epochs=300)

Epoch 1/300
16/16 [==============================] - 8s 5ms/step - loss: 368.1548
Epoch 2/300
16/16 [==============================] - 0s 7ms/step - loss: 69.1719
Epoch 3/300
16/16 [==============================] - 0s 7ms/step - loss: 35.8660
Epoch 4/300
16/16 [==============================] - 0s 6ms/step - loss: 25.5460
Epoch 5/300
16/16 [==============================] - 0s 6ms/step - loss: 20.6187
Epoch 6/300
16/16 [==============================] - 0s 9ms/step - loss: 16.5406
Epoch 7/300
16/16 [==============================] - 0s 5ms/step - loss: 14.5093
Epoch 8/300
16/16 [==============================] - 0s 6ms/step - loss: 14.4955
Epoch 9/300
16/16 [==============================] - 0s 5ms/step - loss: 12.6241
Epoch 10/300
16/16 [==============================] - 0s 7ms/step - loss: 11.4817
Epoch 11/300
16/16 [==============================] - 0s 6ms/step - loss: 10.7958
Epoch 12/300
16/16 [==============================] - 0s 6ms/step - loss: 10.6961
Epoch 13/300
16/16 [====

Finally, we use the trained model to predict the testing cases and print out the predictions and their MSE

In [ ]:
predictions = model.predict(X_test)[:, 0]
print(predictions)

1/1 [==============================] - 0s 72ms/step
[17.298866 19.584328 22.703295 17.573812 20.238115 28.322021 25.970156
 31.649927 29.616001 25.654554]


In [ ]:
print(np.mean((y_test- predictions) ** 2))

39.266244167976694


In [ ]:
model = keras.Sequential([
    keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers. Dropout (0.5),
    keras.layers. Dense (units=1)
])

model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(0.02))
model.fit(X_train, y_train, epochs=300)

Epoch 1/300
16/16 [==============================] - 2s 5ms/step - loss: 417.3208
Epoch 2/300
16/16 [==============================] - 0s 4ms/step - loss: 151.9283
Epoch 3/300
16/16 [==============================] - 0s 4ms/step - loss: 93.2186
Epoch 4/300
16/16 [==============================] - 0s 4ms/step - loss: 71.3338
Epoch 5/300
16/16 [==============================] - 0s 4ms/step - loss: 71.6073
Epoch 6/300
16/16 [==============================] - 0s 4ms/step - loss: 67.7486
Epoch 7/300
16/16 [==============================] - 0s 4ms/step - loss: 55.7361
Epoch 8/300
16/16 [==============================] - 0s 5ms/step - loss: 54.1583
Epoch 9/300
16/16 [==============================] - 0s 4ms/step - loss: 53.7708
Epoch 10/300
16/16 [==============================] - 0s 4ms/step - loss: 47.9167
Epoch 11/300
16/16 [==============================] - 0s 4ms/step - loss: 47.0688
Epoch 12/300
16/16 [==============================] - 0s 5ms/step - loss: 42.9942
Epoch 13/300
16/16 [===

In [ ]:
predictions = model.predict(X_test)[:, 0]
print(predictions)

1/1 [==============================] - 0s 42ms/step
[15.709299 17.320017 19.403528 17.028322 18.081177 22.700413 20.760109
 26.374037 23.954906 20.063343]


In [ ]:
print(np.mean((y_test- predictions) ** 2))

9.867583514946528


# Predicting stock prices with neural networks

We will build the stock predictor with TensorFlow in this section. We will start with feature generation and data preparation, followed by network building and training. After that, we will fine-tune the network and incorporate early stopping to boost the stock predictor.

We can load the data we just downloaded as follows

In [ ]:
import pandas as pd
import numpy as np

from  google.colab import drive
drive.mount('/content/drive')

mydata = pd.read_csv("/content/drive/My Drive/Colab Notebooks/20051201_20051210.csv", index_col="Date")
mydata

Next we define some functions to transform our data

In [ ]:
def add_original_feature(df, df_new):
  df_new['open'] = df['Open']
  df_new['open_1'] = df['Open'].shift(1)
  df_new['close_1'] = df['Close'].shift(1)
  df_new['high_1'] = df['High'].shift(1)
  df_new['low_1'] = df['Low'].shift(1)
  df_new['volume_1'] = df['Volume'].shift(1)

def add_avg_price(df, df_new):
  df_new['avg_price_5'] = df['Close'].rolling(5).mean().shift(1)
  df_new['avg_price_30'] = df['Close'].rolling(21).mean().shift(1)
  df_new['avg_price_365'] = df['Close'].rolling(252).mean().shift(1)
  df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
  df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
  df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']

def add_avg_volume(df, df_new):
  df_new['avg_volume_5'] = df['Volume'].rolling(5).mean().shift(1)
  df_new['avg_volume_30'] = df['Volume'].rolling(21).mean().shift(1)
  df_new['avg_volume_365'] = df['Volume'].rolling(252).mean().shift(1)

  df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
  df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
  df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']

def add_std_price(df, df_new):
  df_new['std_price_5'] = df['Close'].rolling(5).std().shift(1)
  df_new['std_price_30'] = df['Close'].rolling(21).std().shift(1)
  df_new['std_price_365'] = df['Close'].rolling(252).std().shift(1)
  df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
  df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
  df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']

def add_std_volume(df, df_new):
  df_new['std_volume_5'] = df['Volume'].rolling(5).std().shift(1)
  df_new['std_volume_30'] = df['Volume'].rolling(21).std().shift(1)
  df_new['std_volume_365'] = df['Volume'].rolling(252).std().shift(1)
  df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
  df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
  df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']

def add_return_feature(df, df_new):
  df_new['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
  df_new['return_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
  df_new['return_30'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
  df_new['return_365'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)
  df_new['moving_avg_5'] = df_new['return_1'].rolling(5).mean().shift(1)
  df_new['moving_avg_30'] = df_new['return_1'].rolling(21).mean().shift(1)
  df_new['moving_avg_365'] = df_new['return_1'].rolling(252).mean().shift(1)

def generate_features (df):
  """
  Generate features for a stock/index based on historical price and performance
  @param df: dataframe with columns "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
  @return: dataframe, data set with new features
  """
  df_new = pd.DataFrame()
  # 6 original features
  add_original_feature(df, df_new)
  # 31 generated features
  add_avg_price(df, df_new)
  add_avg_volume (df, df_new)
  add_std_price(df, df_new)
  add_std_volume(df, df_new)
  add_return_feature(df, df_new)
  # the target
  df_new['close'] = df['Close']
  df_new = df_new.dropna(axis=0)
  return df_new

In [ ]:
data_raw = pd.read_csv("/content/drive/My Drive/Colab Notebooks/19880101_20191231.csv", index_col="Date")
data = generate_features(data_raw)
print(data.round(decimals=3).head(5))

# Training a simple neural network

We construct the training set using data from 1988 to 2018 and the testing set using data from 2019

In [ ]:
data_raw = pd.read_csv("/content/drive/My Drive/Colab Notebooks/19880101_20191231.csv", index_col="Date")
data = generate_features(data_raw)
data

In [ ]:
start_train = '1988-01-01'
end_train = '2018-12-31'
start_test = '2019-01-01'
end_test = '2019-12-31'
data_train = data.loc[start_train:end_train]
X_train = data_train.drop('close', axis=1).values
y_train = data_train['close'].values
data_test = data.loc[start_test:end_test]
X_test = data_test.drop('close', axis=1).values
y_test = data_test['close'].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.fit_transform(X_test)


We now build a neural network model using the Keras Sequential API

After defining the model, we train it against the training set

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(units=32, activation='relu'),
    Dense(units=1)
])
model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(0.1))
model.fit(X_scaled_train, y_train, epochs=100, verbose=True)

Finally, we use the trained model to predict the testing data and display metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

predictions = model.predict(X_scaled_test)
print(f'MSE: {mean_squared_error(y_test,predictions):.3f}')
print(f'MAE: {mean_absolute_error(y_test, predictions):.3f}')
print(f'R^2: {r2_score(y_test, predictions):.3f}')


# Fine-tuning the neural network

We perform model fine-tuning in TensorFlow

After initializing the hyperparameters to optimize, we create a function to train and validate the model that will take the hyperparameters as arguments, and output the performance

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_HIDDEN = hp.HParam('hidden_size', hp.Discrete([64, 32, 16]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([300, 1000]))
HP_LEARNING_RATE= hp.HParam('learning_rate', hp.RealInterval(0.01, 0.4))

def train_test_model(hparams, logdir):
  model = Sequential([
      Dense(units=hparams[HP_HIDDEN], activation='relu'),
      Dense(units=1)
  ])
  model.compile(
      loss='mean_squared_error',
      optimizer=tf.keras.optimizers.Adam(hparams[HP_LEARNING_RATE]),
      metrics=['mean_squared_error']
  )
  model.fit(X_scaled_train, y_train,
            validation_data=(X_scaled_test, y_test),
            epochs=hparams [HP_EPOCHS],
            verbose=False,
            callbacks=[
              tf.keras.callbacks. TensorBoard (logdir), hp.KerasCallback(logdir, hparams),
              tf.keras.callbacks.EarlyStopping(
              monitor= 'val_loss', min_delta=0, patience=200, verbose=0,
              mode= 'auto',
              )
            ],
  )
  mse = model.evaluate(X_scaled_test, y_test)
  pred = model.predict(X_scaled_test)
  r2 = r2_score (y_test, pred)
  return mse, r2

def run(hparams, logdir):
  with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_HIDDEN, HP_EPOCHS, HP_LEARNING_RATE],
        metrics=[hp.Metric('mean_squared_error', display_name='mse'),
                 hp.Metric('r2', display_name='r2')],
    )
    mse, r2 = train_test_model(hparams, logdir)
    tf.summary.scalar('mean_squared_error', mse, step=1)
    tf.summary.scalar('r2', r2, step=1)

In [ ]:
# session_num = 0
# for hidden in HP_HIDDEN.domain.values:
#   for epochs in HP_EPOCHS.domain.values:
#     for learning_rate in tf.linspace(HP_LEARNING_RATE.domain.min_value,HP_LEARNING_RATE.domain.max_value,5):
#       hparams  = {
#           HP_HIDDEN: hidden,
#           HP_EPOCHS: epochs,
#           HP_LEARNING_RATE:
#             float("%.2f"%float(learning_rate)),
#       }
#       run_name = "run-%d" % session_num
#       print('-- Starting trial: %s' % run_name)
#       print({h.name: hparams[h] for h in hparams})
#       run (hparams, 'logs/hparam_tuning/' + run_name)
#       session_num += 1a

Finally, we use the optimal model to make predictions

In [ ]:
model = Sequential([
    Dense (units=16, activation='relu'),
    Dense (units=1)
])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers. Adam(0.21))
model.fit(X_scaled_train, y_train, epochs=1000, verbose=False)
predictions = model.predict(X_scaled_test)[:, 0]

Plot the prediction along with the ground truth

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data_test.index, y_test, c='k')
plt.plot(data_test.index, predictions, c='b')
plt.plot(data_test.index, predictions, c='r')
plt.plot(data_test.index, predictions, c='g')
plt.xticks(range(0, 252, 10), rotation=60)
plt.xlabel('Date')
plt.ylabel('Close price')
plt.legend(['Truth', 'Neural network prediction'])
plt.show()